# two and more base tracks away correct estimation

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
df_test = pd.read_csv('DS_2_test.csv', index_col=0)

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
zl = np.unique(df['Z'])
zmin = np.min(zl)
zmax = np.max(zl)
dz = np.diff(zl)[0]

In [12]:
def dmin(v, vl): 
    1/0
    return np.min(np.linalg.norm(v.values - vl, axis=1))
def ddmin_f(efr, df, n):
    z0 = efr.iloc[0]['Z']
    if z0 >= zmax - (n-1)*dz:
        efr['dfore'] = 1e6
    else:
        eto = df.query("Z=={}".format(z0 + n*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "XE", "YE"]]
        d = gfr.apply(dmin, axis=1, args=(gto.values,))
        efr['dfore'] = d
    return efr
def ddmin_b(eto, df, n):
    z0 = eto.iloc[0]['Z']
    if z0 <= zmin + (n-1)*dz:
        eto['dback'] = 1e6
    else:
        efr = df.query("Z=={}".format(z0 - n*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "XE", "YE"]]
        d = gto.apply(dmin, axis=1, args=(gfr.values,))
        eto['dback'] = d
    return eto
def ddmin(ev, df, n):
    return ddmin_b(ddmin_f(ev, df, n), df, n)

In [13]:
def features(df, n):
    dnum = np.sqrt(1 - df.TX**2 - df.TY**2)
    dx = df.TX / dnum * dz
    dy = df.TY / dnum * dz
    df['XE'] = df['X'] + dx
    df['YE'] = df['Y'] + dy
    df['X1'] = df['X'] + (n-1) * dx
    df['Y1'] = df['Y'] + (n-1) * dy
    df['X2'] = df['X1'] + n * dx
    df['Y2'] = df['Y1'] + n * dy
    return df.groupby("Z").apply(ddmin, df, n)

In [6]:
print(len(df.query("signal==1")), len(df.query("signal==0")))

162209 9649400


In [7]:
def base_n_away(n):
    rename = {"dfore":"dfore{}".format(n), "dback":"dback{}".format(n)}
    xdf = df.groupby("brick_number").apply(features, n)
    xdfo = xdf[['dfore', 'dback']].rename(columns=rename)
    xdfo.to_hdf("/data/episode2/nata-r{}.h5".format(n), "train")
    xdft = df_test.groupby("brick_number").apply(features, n)
    xdfot = xdft[['dfore', 'dback']].rename(columns=rename)
    xdfot.to_hdf("/data/episode2/nata-r{}.h5".format(n), "test")
    return n

In [8]:
from concurrent.futures import ProcessPoolExecutor

In [9]:
# with ProcessPoolExecutor(max_workers=6) as executor:
#     rst = list(executor.map(base_n_away, (1,2,3,4,5,6)))

In [14]:
n=1
df0 = df.query("brick_number==53")
dnum = np.sqrt(1 - df0.TX**2 - df0.TY**2)
dx = df0.TX / dnum * dz
dy = df0.TY / dnum * dz
df0['XE'] = df0['X'] + dx
df0['YE'] = df0['Y'] + dy
df0['X1'] = df0['X'] + (n-1) * dx
df0['Y1'] = df0['Y'] + (n-1) * dy
df0['X2'] = df0['X1'] + n * dx
df0['Y2'] = df0['Y1'] + n * dy
z0 = 65943
zf = df0.query("Z=={}".format(z0))
eto = df0.query("Z=={}".format(z0 + n*dz))
ddmin(zf, df0, 1)

/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in sqrt
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCop

ZeroDivisionError: ('division by zero', 'occurred at index 0')

In [15]:
%debug

> <ipython-input-12-205401570b4f>(2)dmin()
      1 def dmin(v, vl):
----> 2     1/0
      3     return np.min(np.linalg.norm(v.values - vl, axis=1))
      4 def ddmin_f(efr, df, n):
      5     z0 = efr.iloc[0]['Z']

ipdb> print v
*** SyntaxError: Missing parentheses in call to 'print'
ipdb> print(v)
X1    50550.007812
Y1    66924.796875
X2    51631.557862
Y2    66777.895418
Name: 0, dtype: float64
ipdb> print(vl)
[[ 30078.1953125   69650.3984375   30807.52274433  68779.8024566 ]
 [ 65242.796875    13197.4101562   64920.91612212  13634.41413953]
 [ 35915.625       38897.3046875   36559.43590568  38250.75617329]
 ..., 
 [ 60513.171875    60417.1640625   60207.93287561  59723.78371622]
 [ 31923.2597656   58741.1171875   32482.8113096   59657.04254606]
 [ 70042.0234375   16945.9296875   69406.49039212  16675.75724954]]
ipdb> np.min(np.linalg.norm(v.values - vl, axis=1))
nan
ipdb> v.values
array([ 50550.0078125 ,  66924.796875  ,  51631.55786239,  66777.89541794])
ipdb> np.linalg.norm(v.va

In [18]:
df0[np.isnan(df0['XE'])]

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,XE,YE,X1,Y1,X2,Y2
index,,,,,,,,,,,,,,,
1187840,11557,56089.835938,44951.562500,46548.0,0.678464,0.856505,0.942991,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
1198003,32315,41432.054688,49184.734375,63357.0,0.655032,0.936160,0.623955,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
4127980,11557,57825.664062,45100.960938,68529.0,0.905884,0.544089,2.326501,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
6621868,35130,47891.062500,64902.027344,60771.0,-0.872129,-0.721373,0.364179,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
7372418,35130,49770.945312,64226.804688,27153.0,-0.649674,0.780255,2.540789,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
7424026,35130,50667.125000,65771.187500,59478.0,-0.740699,-0.803720,2.115608,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
8016841,32315,39144.410156,49525.476562,67236.0,0.860533,0.843171,0.514506,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
8023799,35130,49434.109375,65775.789062,51720.0,-0.959852,-0.571066,0.859869,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
8537844,11557,54822.921875,44031.457031,43962.0,0.693564,0.809738,0.781377,1.0,53,NaN,NaN,NaN,NaN,NaN,NaN
